# Consolidate Output
This program reads in the data from runs on the BlueBEAR cluster via RDS and appends them on to the existing `malatium` and `copper` dataframes, saving them as `atium` and `bronze` respectively.

In [1]:
import numpy as np
import lightkurve as lk
import pandas as pd
import fnmatch as fnm
import seaborn as sns
import astropy.units as u
from tqdm import tqdm
import glob
import matplotlib.pyplot as plt

### Add the parameters results

Read in the existing dataframes

In [2]:
cols = ['KIC','Teff','age','dnu','eTeff','ednu','efeh','enumax','feh','loage','lomodlogg','lomodmass','lomodrad','modlogg','modmass','modrad','numax','source','upage','upmodlogg', 'upmodmass','upmodrad','G','bprp']
mal = pd.read_csv('../../data/malatium.csv', usecols=cols)

In [3]:
#Add the necessary columns to malatium
mal['nus*'] = np.nan
mal['l_nus*'] = np.nan
mal['u_nus*'] = np.nan

mal['cosi'] = np.nan
mal['l_cosi'] = np.nan
mal['u_cosi'] = np.nan

mal['i'] = np.nan
mal['l_i'] = np.nan
mal['u_i'] = np.nan

mal['nus'] = np.nan
mal['l_nus'] = np.nan
mal['u_nus'] = np.nan

mal['P'] = np.nan
mal['l_P'] = np.nan
mal['u_P'] = np.nan

mal['flag'] = ''

Parse all the data, including some flags:

`empty   : run not yet done`

`rhat_01 : has at least one rhat > 1.01. Minor concern`

`rhat_1  : has at least one rhat > 1.1. Major concern, exclude from sample`

`nosum   : there is no summary available for some reason`

In [4]:
for idx in range(95):
    kic = mal.loc[idx].KIC
    files = glob.glob('/home/oliver/PhD/mnt/RDS/malatium/peakbag/{}/*chains.csv'.format(str(kic)))

    try:
        chains = pd.read_csv(files[0],index_col=0)
    except IndexError:
        print('Star {} has not completed yet | idx {}'.format(str(kic), idx))
        mal.loc[idx, 'flag'] = 'empty'
        continue

    #Before we go ahead, lets check the Rhats to see if we need to flag anything
    try:
        summary = pd.read_csv(files[0].replace('chains','summary'), index_col=0)
        rhats = np.array([summary.loc['xsplit'].Rhat, summary.loc['cosi'].Rhat,
                         summary.loc['i'].Rhat,summary.loc['split'].Rhat, 
                         summary.loc['b'].Rhat])
        
        if any(np.abs(rhats - 1.) >= 0.01):
            mal.loc[idx, 'flag'] = 'rhat_01'  
        
        if any(np.abs(rhats - 1.) >= 0.1):
            print('Rhats greater than 1.1 for KIC {}'.format(str(kic)))
            mal.loc[idx, 'flag'] = 'rhat_1'

        
    except FileNotFoundError:
        print('No summary file for KIC {}'.format(str(kic)))
        mal.loc[idx, 'flag'] = 'nosum'
        pass

    mal.loc[idx, 'l_nus*'], mal.loc[idx, 'nus*'], mal.loc[idx, 'u_nus*'] = np.percentile(chains['xsplit'].values,
                                                                                         [15.9, 50, 84.1])
    mal.loc[idx, 'l_cosi'], mal.loc[idx, 'cosi'], mal.loc[idx, 'u_cosi'] = np.percentile(chains['cosi'].values,
                                                                                         [15.9, 50, 84.1])
    mal.loc[idx, 'l_nus'], mal.loc[idx, 'nus'], mal.loc[idx, 'u_nus'] = np.percentile(chains['split'].values,
                                                                                      [15.9, 50, 84.1])
    mal.loc[idx, 'l_i'], mal.loc[idx, 'i'], mal.loc[idx, 'u_i'] = np.percentile(chains['i'].values,
                                                                                [15.9, 50, 84.1])    
    
    nus = u.Quantity(chains['split'].values, u.microhertz)
    Pchain = 1./u.Quantity(nus, 1./u.day).value
    mal.loc[idx, 'l_P'], mal.loc[idx, 'P'], mal.loc[idx, 'u_P'] = np.percentile(Pchain, [15.9, 50, 84.1])  

Rhats greater than 1.1 for KIC 5950854
Rhats greater than 1.1 for KIC 6603624
Star 7510397 has not completed yet | idx 22
Star 7970740 has not completed yet | idx 27
No summary file for KIC 8006161
No summary file for KIC 8379927
Star 8394589 has not completed yet | idx 33
Star 8760414 has not completed yet | idx 36
Rhats greater than 1.1 for KIC 8938364
Star 9139163 has not completed yet | idx 41
Star 9206432 has not completed yet | idx 42
Star 9414417 has not completed yet | idx 45
No summary file for KIC 9965715
Star 10516096 has not completed yet | idx 53
Rhats greater than 1.1 for KIC 10730618
Star 11253226 has not completed yet | idx 58
Star 12069424 has not completed yet | idx 62
Star 12069449 has not completed yet | idx 63
Rhats greater than 1.1 for KIC 6521045
Star 8478994 has not completed yet | idx 81
No summary file for KIC 8866102


In [5]:
mal.head(5)

,KIC,Teff,age,dnu,eTeff,ednu,efeh,enumax,feh,loage,...,i,l_i,u_i,nus,l_nus,u_nus,P,l_P,u_P,flag
0,1435467,6326,3.0164,70.369,77,0.034,0.1,6.3,0.01,0.3513,...,1.178398,1.064167,1.251533,1.671453,1.593461,1.813538,6.924557,6.382040,7.263483,
1,2837475,6614,1.6270,75.729,77,0.041,0.1,8.2,0.01,0.1757,...,1.199428,1.166976,1.233080,3.345075,3.283668,3.412607,3.460034,3.391563,3.524739,
2,3427720,6045,2.2339,120.068,77,0.031,0.1,10.7,-0.06,0.2396,...,0.891136,0.570728,1.303701,0.424040,0.320092,0.693356,27.294770,16.692830,36.158605,
3,3456181,6384,2.0901,52.264,77,0.041,0.1,8.3,-0.15,0.1278,...,0.577564,0.539859,0.614492,2.126247,2.023164,2.228384,5.443428,5.193932,5.720778,
4,3632418,6193,2.6331,60.704,77,0.019,0.1,3.0,-0.12,0.1757,...,1.217700,1.106743,1.386214,1.034966,0.972231,1.100373,11.183052,10.518321,11.904653,


In [6]:
mal.to_csv('../../data/atium.csv')

### Add the mode results

Read in the existing dataframe

In [7]:
cop = pd.read_csv('../../data/copper.csv', index_col=0)

Parse all the data, including some flags:

`empty   : run not yet done`

`rhat_1  : has at least one rhat > 1.1. Major concern, exclude from sample`

`nosum   : there is no summary available for some reason`


In [8]:
cop['f'] = np.nan
cop['f_e'] = np.nan
cop['g'] = np.nan
cop['g_e'] = np.nan
cop['A'] = np.nan
cop['A_e'] = np.nan
cop['H'] = np.nan
cop['H_e'] = np.nan
cop['flag'] = ''

In [9]:
for idx in range(95):
    kic = str(mal.loc[idx].KIC)
    files = glob.glob('/home/oliver/PhD/mnt/RDS/malatium/peakbag/{}/*chains.csv'.format(str(kic)))

    try:
        chains = pd.read_csv(files[0],index_col=0)
        lis = list(chains)
    except IndexError:
        print('Star {} has not completed yet | idx {}'.format(str(kic), idx))
        cop.loc[cop.KIC == kic, 'flag'] = 'empty'
        continue

    # Lets do each mode in turn
    for mode in [0,1,2]:
        cop.loc[(cop.KIC==kic)&(cop.l==mode),'f'] = np.median(chains[fnm.filter(lis, 'f{}_*'.format(mode))], axis=0)
        cop.loc[(cop.KIC==kic)&(cop.l==mode),'f_e'] = np.std(chains[fnm.filter(lis, 'f{}_*'.format(mode))], axis=0).values
        cop.loc[(cop.KIC==kic)&(cop.l==mode),'g'] = np.median(chains[fnm.filter(lis, 'g{}_*'.format(mode))], axis=0)
        cop.loc[(cop.KIC==kic)&(cop.l==mode),'g_e'] = np.std(chains[fnm.filter(lis, 'g{}_*'.format(mode))], axis=0).values
        cop.loc[(cop.KIC==kic)&(cop.l==mode),'A'] = np.median(chains[fnm.filter(lis, 'a{}_*'.format(mode))], axis=0)
        cop.loc[(cop.KIC==kic)&(cop.l==mode),'A_e'] = np.std(chains[fnm.filter(lis, 'a{}_*'.format(mode))], axis=0).values
        cop.loc[(cop.KIC==kic)&(cop.l==mode),'H'] = np.median(chains[fnm.filter(lis, 'h{}_*'.format(mode))], axis=0)
        cop.loc[(cop.KIC==kic)&(cop.l==mode),'H_e'] = np.std(chains[fnm.filter(lis, 'h{}_*'.format(mode))], axis=0).values    
    
    #Lets copy over the Rhats for each mode and flag if needed
    try:
        summary = pd.read_csv(files[0].replace('chains','summary'), index_col=0)
        rhats = summary.loc[fnm.filter(lis, '*__*')].Rhat.values
        
        if any(np.abs(rhats - 1.) >= 0.1):
            print('Rhats greater than 1.1 for KIC {}'.format(str(kic)))
            cop.loc[cop.KIC==kic, 'flag'] = 'rhat_1'

        
    except FileNotFoundError:
        print('No summary file for KIC {}'.format(str(kic)))
        mal.loc[idx, 'flag'] = 'nosum'
        pass

Rhats greater than 1.1 for KIC 3656476
Rhats greater than 1.1 for KIC 6106415
Rhats greater than 1.1 for KIC 6603624
Star 7510397 has not completed yet | idx 22
Star 7970740 has not completed yet | idx 27
No summary file for KIC 8006161
No summary file for KIC 8379927
Star 8394589 has not completed yet | idx 33
Rhats greater than 1.1 for KIC 8424992
Star 8760414 has not completed yet | idx 36
Star 9139163 has not completed yet | idx 41
Star 9206432 has not completed yet | idx 42
Star 9414417 has not completed yet | idx 45
No summary file for KIC 9965715
Rhats greater than 1.1 for KIC 10079226
Star 10516096 has not completed yet | idx 53
Star 11253226 has not completed yet | idx 58
Star 12069424 has not completed yet | idx 62
Star 12069449 has not completed yet | idx 63
Rhats greater than 1.1 for KIC 6521045
Star 8478994 has not completed yet | idx 81
No summary file for KIC 8866102
Rhats greater than 1.1 for KIC 11401755


In [10]:
cop.head(5)

,KIC,n,l,Freq,e_Freq,E_Freq,Amp,e_Amp,E_Amp,Width,...,source,f,f_e,g,g_e,A,A_e,H,H_e,flag
0,1435467,11,1,889.25,0.47,0.53,NaN,NaN,NaN,NaN,...,L,889.771948,0.320578,4.267716,1.218315,2.203952,0.170429,0.728456,0.182340,
1,1435467,11,2,920.04,1.82,1.99,NaN,NaN,NaN,NaN,...,L,921.220902,1.643444,3.067566,1.534018,1.334852,0.324376,0.380058,0.227212,
2,1435467,12,0,928.22,0.94,0.71,2.39,0.14,0.13,4.79,...,L,927.920894,0.616308,3.702034,1.526683,1.383523,0.351100,0.332163,0.147880,
3,1435467,12,1,957.04,0.50,0.46,NaN,NaN,NaN,NaN,...,L,957.393253,0.239691,2.109080,0.636603,2.062365,0.138292,1.282096,0.321852,
4,1435467,12,2,989.14,1.24,1.30,NaN,NaN,NaN,NaN,...,L,989.204494,0.391108,1.597421,0.952271,1.723303,0.221747,1.161414,0.557360,


In [11]:
cop.to_csv('../../data/bronze.csv')